# OdC's 2025 Temperature Geovisor

This notebook loads each city's temperature data, processes and __creates the .htmls needed for kepler visualization.__

* __Important note: Worked fine with keplergl v0.3.2, but not with kepler v.0.3.7.__
* __Final parameters used for Temperature Visor:__
  * __res__: 10
  * __filter_urban__: True
  * __palette__: palette_2025
  * __stroke__: 0.05, white
  * __opacity__: 0.10

In [1]:
# Temperature anomaly 2025 OdC's social media publications (Satellite background)
# Hotter 3: "#ca0020" (Red)
# Hotter 2: "#e66e61" (Light red)
# Hotter 1:"#f5c0a9" (Very light red)
# Middle: "#f7f7f7" (Nearly white)
# Colder 1: "#b4d6e6" (Very light blue)
# Colder 2: "#63a9cf" (Light blue)
# Colder 3: "#0571b0" (Blue)

## Import libraries

In [2]:
from pathlib import Path

current_path = Path().resolve()

for parent in current_path.parents:
    if parent.name == "accesibilidad-urbana":
        project_root = parent
        break

print(project_root)

/home/jovyan/accesibilidad-urbana


In [3]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt

# Imports que venían en el Notebook 15-min-city > 13-15-min-kepler-test.ipynb
import io
#import boto3
from keplergl import KeplerGl

# Classify data using Natural Breaks
import mapclassify

# Correlation calc
import scipy.stats as stats

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join(project_root))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
else:
    import aup

## Config notebook

In [4]:
# Set test True (runs specified city_lst) or False (runs all available cities)
test = False

if test:
    city_lst = ['Aguascalientes'] # Gets overwritten if test = False
    res = 10

    # MANUAL CATEGORIZATION TEST
    # From AI (Short research for manual_categorization tests)
    #high_termal_variability_expected = ['CDMX','ZMVM','Guadalajara','Monterrey','Chihuahua','Queretaro']
    #low_termal_variability_expected = ['Tepic','Merida','Villahermosa','Mazatlan','Oaxaca']
    #city_lst = high_termal_variability_expected + low_termal_variability_expected
    
else:
    res = 10

projected_crs = 'EPSG:6372'
# Processing - Filter for urban areas only?
filter_urban = True
# Processing - If running CDMX (capital city), add ZMVM (metropolitan area)
merge_capital = True
# Processing - Categorize data using manual input? (Else uses jenks)
manual_categorization = True

# Directory where .html files are saved
map_output_dir = str(project_root) + f"/data/processed/visor_temperature/"

## Find all available cities (If test = False, substitutes city_list with all cities available in db)

In [5]:
if test == False:
    # Find all available cities
    temp_schema = 'raster_analysis'
    temp_table = 'temperature_analysis_hex'
    scan_res = 8
    query = f"SELECT city FROM {temp_schema}.{temp_table} WHERE \"res\" = '{scan_res}\'"
    temperature_df = aup.df_from_query(query)
    city_lst = list(temperature_df.city.unique())
    del temperature_df

# If merge_capital, drop ZMVM in order to load ZMVM when loading CDMX during processing.
if merge_capital and ('CDMX' in city_lst) and ('ZMVM' in city_lst):
    print("--"*30)
    print("merge_capital IS SET TO TRUE: City list contains both CDMX and ZMVM. Will merge both databases and create a unified temperature anomaly.")
    print("--"*30)
    city_lst.remove('ZMVM')

print(f"{len(city_lst)} cities to run at res {res}.")
city_lst

------------------------------------------------------------
merge_capital IS SET TO TRUE: City list contains both CDMX and ZMVM. Will merge both databases and create a unified temperature anomaly.
------------------------------------------------------------
49 cities to run at res 10.


['Acapulco',
 'Aguascalientes',
 'CDMX',
 'Chihuahua',
 'Chilpancingo',
 'Ciudad Obregon',
 'Colima',
 'Cordoba',
 'Cuautla',
 'Cuernavaca',
 'Delicias',
 'Durango',
 'Guadalajara',
 'Guanajuato',
 'Guaymas',
 'Irapuato',
 'Juarez',
 'Laguna',
 'La Paz',
 'Leon',
 'Los Cabos',
 'Matamoros',
 'Mazatlan',
 'Merida',
 'Minatitlan',
 'Monterrey',
 'Nuevo Laredo',
 'Oaxaca',
 'Orizaba',
 'Pachuca',
 'Piedad',
 'Piedras Negras',
 'Queretaro',
 'Saltillo',
 'San Martin',
 'SLP',
 'Tapachula',
 'Tehuacan',
 'Tepic',
 'Tijuana',
 'Toluca',
 'Tulancingo',
 'Tuxtla',
 'Vallarta',
 'Veracruz',
 'Victoria',
 'Villahermosa',
 'Zacatecas',
 'Zamora']

## Create Kepler HTMLs files by city

In [6]:
i=0
for city in city_lst:

    try:
        print("--"*30)
        print(f"--- STARTING CITY {i}/{len(city_lst)}: {city}.")
    
        # 1.0 --- --- --- LOAD DATA --- --- ---
        
        # 1.1 --- LOAD URBAN HEXS
        print(f"Loading {city}'s urban hexs.")
        # Load data
        hex_schema = 'hexgrid'
        hex_table = f'hexgrid_{res}_city_2020'
        # Load city's hexs filtering for urban areas if required
        if filter_urban:
            hex_type = 'urban'
            query = f"SELECT hex_id_{res}, geometry FROM {hex_schema}.{hex_table} WHERE \"city\" = '{city}\' AND \"type\" = '{hex_type}\'"
        else:
            query = f"SELECT hex_id_{res}, geometry FROM {hex_schema}.{hex_table} WHERE \"city\" = '{city}\'"
        hex_gdf = aup.gdf_from_query(query, geometry_col='geometry')
        
        # SPECIFIC CASE - Merge capital's hexs (CDMX + ZMVM)
        if merge_capital and (city == 'CDMX'):
            print("MERGING CDMX + ZMVM hexs.")
            # Load ZMVM's hexs filtering for urban areas if required
            if filter_urban:
                hex_type = 'urban'
                query = f"SELECT hex_id_{res}, geometry FROM {hex_schema}.{hex_table} WHERE \"city\" = 'ZMVM\' AND \"type\" = '{hex_type}\'"
            else:
                query = f"SELECT hex_id_{res}, geometry FROM {hex_schema}.{hex_table} WHERE \"city\" = 'ZMVM\'"
            metro_hex_gdf = aup.gdf_from_query(query, geometry_col='geometry')
            # Merge CDMX and ZMVM hexs
            hex_gdf = pd.concat([hex_gdf,metro_hex_gdf])
            
        # Read and format cols
        hex_gdf['res'] = res
        hex_gdf.rename(columns={f'hex_id_{res}':'hex_id'},inplace=True)
        hex_gdf.to_crs(projected_crs,inplace=True)
        # List all unique hex_ids
        hexid_lst = list(hex_gdf.hex_id.unique())
        del hex_gdf
         
        # 1.2 --- LOAD TEMPERATURE DATA
        print(f"Loading {city}'s Temperature data.")
        temp_schema = 'raster_analysis'
        temp_table = 'temperature_analysis_hex'
        query = f"SELECT * FROM {temp_schema}.{temp_table} WHERE \"city\" = '{city}\' AND \"res\" = '{res}\'"
        temperature_gdf = aup.gdf_from_query(query, geometry_col='geometry')

        # SPECIFIC CASE - Merge capital's temperature hexs (CDMX + ZMVM)
        if merge_capital and (city == 'CDMX'):
            print("MERGING CDMX + ZMVM TEMPERATURE.")
            # Load metro area's temperature hexs
            query = f"SELECT * FROM {temp_schema}.{temp_table} WHERE \"city\" = 'ZMVM\' AND \"res\" = '{res}\'"
            metro_temp_gdf = aup.gdf_from_query(query, geometry_col='geometry')
            # Merge capital and metro area temperature hexs
            temperature_gdf = pd.concat([temperature_gdf,metro_temp_gdf])
            # Drop duplicated temperature hexs (Both in CDMX and ZMVM)
            temperature_gdf.drop_duplicates(subset="hex_id",
                                            inplace=True)
        
        temperature_gdf.to_crs(projected_crs,inplace=True)
        # Filter for urban areas if required
        if filter_urban:
            temperature_gdf = temperature_gdf.loc[temperature_gdf.hex_id.isin(hexid_lst)].copy()
        # Inf values check
        infs = temperature_gdf.loc[np.isinf(temperature_gdf['temperature_mean'])]
        if len(infs)>0:
            print(f"WARNING: Dropping {len(infs)} hexs res {res} because of inf values.")
            temperature_gdf = temperature_gdf.loc[~np.isinf(temperature_gdf['temperature_mean'])].copy()
        
        # 2.0 --- --- --- CALCULATIONS AND DATA TREATMENT --- --- ---
        
        # 2.1 --- CALCULATE TEMPERATURE ANOMALY
        print("DATA TREATMENT - Calculating temperature anomaly.")
        # Calculate anomaly by hex (differential between mean in each hex and city mean)
        mean_city_temperature = temperature_gdf.temperature_mean.mean()
        temperature_gdf['temperature_anomaly'] = temperature_gdf['temperature_mean'] - mean_city_temperature
        
        # 2.2 --- CATEGORIZE TEMPERATURE ANOMALY USING PREDEFINED VALUES
        if manual_categorization:
            ########### CATEGORIZATION OPTION 1: SET MANUALLY ###########
            # Define costumized boundaries
            classif_bins = [-100,-3.5,-1.5,-0.5,0.5,1.5,3.5,100] #Edgar
            #classif_bins = [-15, -6.5, -3.5, -1.30, 0.35, 2.0, 4.0, 15] #Based in Monterrey
            # Clasify data in bins
            temperature_gdf['anomaly_class'] = pd.cut(temperature_gdf['temperature_anomaly'],
                                                      bins=classif_bins,
                                                      labels=[-3, -2, -1, 0, 1, 2, 3],
                                                      include_lowest=True).astype(int)
        else:
            ########### CATEGORIZATION OPTION 2: SET USING JENKS ###########
            print("DATA TREATMENT - Categorize temperature anomaly using jenks.")
            # Calculate Natural Breaks (Jenks) cuts (using 7 classes)
            classifier = mapclassify.NaturalBreaks(y=temperature_gdf['temperature_anomaly'], k=7)
            # Add class to each hex
            temperature_gdf['anomaly_class'] = classifier.yb
            # Rename classes to be between -3 (colder) and 3 (hotter)
            temperature_gdf['anomaly_class'] -= 3
            # Extract array of Natural Breaks upper bounds
            classif_bins = classifier.bins
        
        # 2.3 --- SET CATEGORY'S NAME TO BE DISPLAYED ON GEOVISOR (Creates bins columns)
        print("DATA TREATMENT - Renaming categories.")
        # Create a column containing anomaly classes
        # (Used to display the layer using a color palette and show the simbology)
        classes_dict = {3:f"1. Más caliente",
                        2:"2.",
                        1:"3.",
                        0:"4.",
                        -1:"5.",
                        -2:"6.",
                        -3:f"7. Más fresco"
                       }
        temperature_gdf['anomaly_bins'] = temperature_gdf['anomaly_class'].map(classes_dict)
        
        # Create a similar but more detailed column
        # (Used to display detailed information when hovering over hexs)
        decimals = 2 #Number of decimals for temperature bins to be show
        classes_dict_detailed = {3:f"{abs(round(classif_bins[6],decimals))}° o más sobre el promedio.",
                                 2:f"{abs(round(classif_bins[5],decimals))}° a {abs(round(classif_bins[6],decimals))}° sobre el promedio.",
                                 1:f"{abs(round(classif_bins[4],decimals))}° a {abs(round(classif_bins[5],decimals))}° sobre el promedio.",
                                 0:"Cercano al promedio.",
                                 -1:f"{abs(round(classif_bins[3],decimals))}° a {abs(round(classif_bins[2],decimals))}° debajo del promedio.",
                                 -2:f"{abs(round(classif_bins[2],decimals))}° a {abs(round(classif_bins[1],decimals))}° debajo del promedio.",
                                 -3:f"{abs(round(classif_bins[1],decimals))}° o más debajo del promedio."
                                }
        temperature_gdf['anomaly_details'] = temperature_gdf['anomaly_class'].map(classes_dict_detailed)
        
        # 2.4 --- CONVERT TO CATEGORICAL ORDER
        print("DATA TREATMENT - Setting bins as categorical values.")
        # Define order and convert col into ordered category
        temperature_categories = list(classes_dict.values())
        temperature_gdf['anomaly_bins'] = pd.Categorical(temperature_gdf['anomaly_bins'], 
                                                         categories=temperature_categories, 
                                                         ordered=True)
        # Force categorical order
        temperature_gdf.sort_values(by='anomaly_bins', inplace=True)

        # 2.5 --- CONSIDER TEMPERATURE ANOMALY DIRECTLY, ROUNDING VALUE
        temperature_gdf['temperature_anomaly_rounded'] = temperature_gdf['temperature_anomaly'].round(2)
        
        # 3.0 --- --- --- PREPARE GDF FOR KEPLER CONFIGURATION --- --- ---

        # 3.1 --- RENAME COLUMNS AND FILTER COLUMNS OF INTEREST
        print("GDF PREPARATION - Preparing hex_kepler.")
        # Rename columns of interest
        main_col = f'Diferencia de temperatura:' #Column with temperature anomaly classified
        hex_anomaly_col = "Diferencia de temperatura con respecto al promedio en la ciudad:" # Column with specific hex data
        details_col = f'Rango de diferencia de temperatura:'
        rename_columns = {'anomaly_bins':main_col,
                          'temperature_anomaly_rounded':hex_anomaly_col,
                          'anomaly_details':details_col}
        hex_kepler = temperature_gdf.copy()
        hex_kepler.rename(columns=rename_columns,inplace=True)
        # Keep columns of interest and geometry
        hex_kepler = hex_kepler[[main_col,details_col,hex_anomaly_col,'geometry']]

        # 3.2 --- SET CITY SIZE (DEFINES HTML ZOOM)
        

        # PREPARE HEX KEPLER - Kepler not loading if it is projected_crs
        if hex_kepler.crs != "EPSG:4326":
            hex_kepler.to_crs("EPSG:4326",inplace=True)
            print(f"GDF PREPARATION - Changed {city}'s hex_kepler crs to EPSG:4326.")
        
        # 4.0 --- --- --- KEPLER CONFIGURATION --- --- ---
        # CREATE CONFIGURATION
        print(f"KEPLER CONFIGURATION - Starting {city}'s map config.")
        config, config_idx = aup.kepler_config()

        # LAYER CONFIGURATION - Set layer to be visualized on map
        print("Layer configuration.")
        config["config"]["visState"]["layers"][0]["visualChannels"]["colorField"]["name"] = main_col
        
        # PALETTE CONFIGURATION - Set the color palette of the visualized layer
        print("Palette configuration.")
        # Temperature anomaly 2025 OdC's social media publications (Satellite background)
        sm2025_palette = ["#ca0020", # Red
                          "#e66e61", # Light red
                          "#f5c0a9", # Very light red
                          "#f7f7f7", # Nearly white
                          "#b4d6e6", # Very light blue
                          "#63a9cf", # Light blue
                          "#0571b0"] # Blue
        # Palette designed to be more compatible with Kepler's satellite background
        darker_palette = ["#8b0000", # Dark red
                          "#ca0020", # Red
                          "#e66e61", # Light red
                          "#f7f7f7", # Nearly white
                          "#63a9cf", # Light blue
                          "#0571b0", # Blue
                          "#034078"] # Dark blue
        
        config["config"]["visState"]["layers"][0]["config"]["visConfig"]["colorRange"] = {"name": "sm2025_palette",
                                                                                          "type": "custom",
                                                                                          "category": "Custom",
                                                                                          "colors": darker_palette,
                                                                                          "reversed": False}
        # STROKE CONFIGURATION - Set stroke width and color [Final decision: 0.05 white]
        print("Stroke configuration.")
        # Black --> "#000000" --> [0, 0, 0]
        # White --> "#ffffff" --> [255, 255, 255]
        stroke=0.05
        strokecolor = [255, 255, 255]
        config["config"]["visState"]["layers"][0]["config"]["visConfig"]["thickness"] = stroke #aup.kepler_config() value: 0.5
        config["config"]["visState"]["layers"][0]["config"]["visConfig"]["strokeColor"] = strokecolor #aup.kepler_config() value: [28, 27, 27]

        # OPACITY CONFIGURATION - Set object's opacity [Final decision: 0.10]
        print("Opacity configuration.")
        fillopacity = 0.05
        config["config"]["visState"]["layers"][0]["config"]["visConfig"]["opacity"] = fillopacity #aup.kepler_config() value: 0.85
        
        # FUNCTIONALITIES CONFIGURATION - Turning on/off map functionalities
        print("Functionalities configuration.")
        # Turn on geocoder (Search tab)
        config["config"]["visState"]["interactionConfig"]["geocoder"]["enabled"] = True
        # Stop user from activating manually dual map viewer
        config["config"]["splitMaps"] = []
        # Force turn on legend
        config["config"]["visState"]["legend"] = {"active": True}
        # FUNCTIONALITIES CONFIGURATIONS ALREADY TURNED OFF IN FUNCTION (Written here as reminder)
        # Turn off dual map viewer #aup.kepler_config() value:False
        #config["config"]["mapState"]["isSplit"] = False
        # Turn off brush #aup.kepler_config() value:False
        #config["config"]["visState"]["interactionConfig"]["brush"]["enabled"] = False
        # Turn off coordinate #aup.kepler_config() value:False
        #config["config"]["visState"]["interactionConfig"]["coordinate"]["enabled"] = False
        
        # STARTING POINT CONFIGURATION - Set map's starting point for each city
        print("Starting point configuration.")
        longitude = hex_kepler.dissolve().geometry.centroid.x
        latitude = hex_kepler.dissolve().geometry.centroid.y
        config["config"]["mapState"]["latitude"] = latitude[0]
        config["config"]["mapState"]["longitude"] = longitude[0]

        # LAYER CONFIGURATION - Set starting zoom for city depending on how big the city is
        # Create buffer around hexagons
        poly = hex_kepler.to_crs(projected_crs).buffer(500)
        poly = gpd.GeoDataFrame(geometry=poly).dissolve().geometry
        # Extract coordinates from polygon
        coord_val = poly.bounds
        n = coord_val.maxy.max()
        s = coord_val.miny.min()
        e = coord_val.maxx.max()
        w = coord_val.minx.min()
        # Find large_boundary of current city
        if abs(n-s) > abs(e-w):
            large_boundary = abs(n-s)
        else:
            large_boundary = abs(e-w)
        print(f"City's large boundary: {large_boundary}.")
        # Define zoom
        if large_boundary >= 75000:
            #aup.kepler_config() value: 8.515158481972351
            zoom = 8.515158481972351
        elif (large_boundary >= 60000) and (large_boundary < 75000):
            zoom = 9
        elif (large_boundary >= 30000) and (large_boundary < 60000):
            zoom = 10
        elif large_boundary < 30000:
            zoom = 11
        config["config"]["mapState"]["zoom"] = zoom
        
        # HOVERING CONFIGURATION - Set data to show when hovering over hex
        print("Hovering configuration.")
        config["config"]["visState"]["interactionConfig"]["tooltip"]["fieldsToShow"] = {"Análisis de hexágono": [hex_anomaly_col,
                                                                                                                 details_col]}  
        
        # END OF CONFIGURATION - Add configuration and data to html file
        print("Add configuration.")
        map_city = KeplerGl(height=800)
        map_city.config = config
        map_city.add_data(hex_kepler, name='Análisis de hexágono')
        
        print(f"KEPLER CONFIGURATION - Finished {city}'s map config.")
        
        # 4.0 --- --- --- SAVE HTML --- --- ---
        if test == True:
            if manual_categorization:
                file_name = f'TestTemperature_{city}_z{zoom}_{res}_op{fillopacity}_strk{stroke}_manual-cat_darkerpalette.html'
            else:
                file_name = f'TestTemperature_{city}_z{zoom}_{res}_op{fillopacity}_strk{stroke}_jenks-cat.html'
            map_city.save_to_html(file_name = map_output_dir+f"tests/{file_name}", read_only=False)
            print(f"SAVING (TEST) - Saved {city}'s map config html.")
        else:
            file_name = f'{city}_temp.html'
            map_city.save_to_html(file_name = map_output_dir+f"{file_name}", read_only=False)
            print(f"SAVING (FINAL) - Saved {city}'s map config html file.")
    
    # In case of error while running city
    except:
        print("--"*30)
        print(f"ERROR: {city}.")
        print("--"*30)

    i+=1

------------------------------------------------------------
--- STARTING CITY 0/49: Acapulco.
Loading Acapulco's urban hexs.
Loading Acapulco's Temperature data.
DATA TREATMENT - Calculating temperature anomaly.
DATA TREATMENT - Renaming categories.
DATA TREATMENT - Setting bins as categorical values.
GDF PREPARATION - Preparing hex_kepler.
GDF PREPARATION - Changed Acapulco's hex_kepler crs to EPSG:4326.
KEPLER CONFIGURATION - Starting Acapulco's map config.
Layer configuration.
Palette configuration.
Stroke configuration.
Opacity configuration.
Functionalities configuration.
Starting point configuration.


/tmp/ipykernel_22707/2243712383.py:233: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  longitude = hex_kepler.dissolve().geometry.centroid.x
/tmp/ipykernel_22707/2243712383.py:234: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  latitude = hex_kepler.dissolve().geometry.centroid.y


City's large boundary: 45669.04076261271.
Hovering configuration.
Add configuration.
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
KEPLER CONFIGURATION - Finished Acapulco's map config.
Map saved to /home/jovyan/accesibilidad-urbana/data/processed/visor_temperature/Acapulco_temp.html!
SAVING (FINAL) - Saved Acapulco's map config html file.
------------------------------------------------------------
--- STARTING CITY 1/49: Aguascalientes.
Loading Aguascalientes's urban hexs.
Loading Aguascalientes's Temperature data.
DATA TREATMENT - Calculating temperature anomaly.
DATA TREATMENT - Renaming categories.
DATA TREATMENT - Setting bins as categorical values.
GDF PREPARATION - Preparing hex_kepler.
GDF PREPARATION - Changed Aguascalientes's hex_kepler crs to EPSG:4326.
KEPLER CONFIGURATION - Starting Aguascalientes's map config.
Layer configuration.
Palette configuration.
Stroke configuration.
Opacity configuration.
Functionalities configuration.
Starting point configuration.


/tmp/ipykernel_22707/2243712383.py:233: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  longitude = hex_kepler.dissolve().geometry.centroid.x
/tmp/ipykernel_22707/2243712383.py:234: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  latitude = hex_kepler.dissolve().geometry.centroid.y


City's large boundary: 47038.56126136449.
Hovering configuration.
Add configuration.
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
KEPLER CONFIGURATION - Finished Aguascalientes's map config.
Map saved to /home/jovyan/accesibilidad-urbana/data/processed/visor_temperature/Aguascalientes_temp.html!
SAVING (FINAL) - Saved Aguascalientes's map config html file.
------------------------------------------------------------
--- STARTING CITY 2/49: CDMX.
Loading CDMX's urban hexs.
MERGING CDMX + ZMVM hexs.
Loading CDMX's Temperature data.
MERGING CDMX + ZMVM TEMPERATURE.
DATA TREATMENT - Calculating temperature anomaly.
DATA TREATMENT - Renaming categories.
DATA TREATMENT - Setting bins as categorical values.
GDF PREPARATION - Preparing hex_kepler.
GDF PREPARATION - Changed CDMX's hex_kepler crs to EPSG:4326.
KEPLER CONFIGURATION - Starting CDMX's map config.
Layer configuration.
Palette configuration.
Stroke configuration.
Opacity configuration.
Functionalities configuration.
Start

...

City's large boundary: 45312.80548431014.
Hovering configuration.
Add configuration.
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
KEPLER CONFIGURATION - Finished Villahermosa's map config.
Map saved to /home/jovyan/accesibilidad-urbana/data/processed/visor_temperature/Villahermosa_temp.html!
SAVING (FINAL) - Saved Villahermosa's map config html file.
------------------------------------------------------------
--- STARTING CITY 47/49: Zacatecas.
Loading Zacatecas's urban hexs.
Loading Zacatecas's Temperature data.
DATA TREATMENT - Calculating temperature anomaly.
DATA TREATMENT - Renaming categories.
DATA TREATMENT - Setting bins as categorical values.
GDF PREPARATION - Preparing hex_kepler.
GDF PREPARATION - Changed Zacatecas's hex_kepler crs to EPSG:4326.
KEPLER CONFIGURATION - Starting Zacatecas's map config.
Layer configuration.
Palette configuration.
Stroke configuration.
Opacity configuration.
Functionalities configuration.
Starting point configuration.


/tmp/ipykernel_22707/2243712383.py:233: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  longitude = hex_kepler.dissolve().geometry.centroid.x
/tmp/ipykernel_22707/2243712383.py:234: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  latitude = hex_kepler.dissolve().geometry.centroid.y


City's large boundary: 31623.702607200015.
Hovering configuration.
Add configuration.
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
KEPLER CONFIGURATION - Finished Zacatecas's map config.
Map saved to /home/jovyan/accesibilidad-urbana/data/processed/visor_temperature/Zacatecas_temp.html!
SAVING (FINAL) - Saved Zacatecas's map config html file.
------------------------------------------------------------
--- STARTING CITY 48/49: Zamora.
Loading Zamora's urban hexs.
Loading Zamora's Temperature data.
DATA TREATMENT - Calculating temperature anomaly.
DATA TREATMENT - Renaming categories.
DATA TREATMENT - Setting bins as categorical values.
GDF PREPARATION - Preparing hex_kepler.
GDF PREPARATION - Changed Zamora's hex_kepler crs to EPSG:4326.
KEPLER CONFIGURATION - Starting Zamora's map config.
Layer configuration.
Palette configuration.
Stroke configuration.
Opacity configuration.
Functionalities configuration.
Starting point configuration.


/tmp/ipykernel_22707/2243712383.py:233: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  longitude = hex_kepler.dissolve().geometry.centroid.x
/tmp/ipykernel_22707/2243712383.py:234: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  latitude = hex_kepler.dissolve().geometry.centroid.y


City's large boundary: 20806.57631748088.
Hovering configuration.
Add configuration.
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
KEPLER CONFIGURATION - Finished Zamora's map config.
Map saved to /home/jovyan/accesibilidad-urbana/data/processed/visor_temperature/Zamora_temp.html!
SAVING (FINAL) - Saved Zamora's map config html file.
